# Transform Smithy apis into MCP tools using Bedrock AgentCore Gateway

## 개요
Bedrock AgentCore Gateway는 고객이 인프라나 호스팅을 관리할 필요 없이 기존 Smithy API를 완전 관리형 MCP 서버로 전환할 수 있는 방법을 제공합니다. 고객은 Smithy 사양을 가져와서 MCP 도구로 변환할 수 있습니다. Amazon S3의 Smithy 모델에서 MCP 도구를 생성하는 방법을 시연할 것입니다. 그러면 에이전트가 Amazon S3를 쿼리하고 관련 질문에 답할 수 있게 됩니다.
Gateway 워크플로우는 에이전트를 외부 도구에 연결하기 위해 다음 단계를 포함합니다:* **Create the tools for your Gateway** - Define your tools using Smithy specification. 
* **Create a Gateway endpoint** - Create the gateway that will serve as the MCP entry point with inbound authentication.
* **Add targets to your Gateway** - Configure the Smithy target that define how the gateway routes requests to specific tools. All the operations that part of Smithy file will become an MCP-compatible tool, and will be made available through your Gateway endpoint URL. Configure outbound authorization using AWS IAM for invoking Amazon S3 apis via Smithy.
* **Update your agent code** - Connect your agent to the Gateway endpoint to access all configured tools through the unified MCP interface.

![작동 방식](images/smithy-apis-gateway.png)

### 튜토리얼 세부 정보


| 정보                 | 세부 사항                                                 |
|:---------------------|:----------------------------------------------------------|
| 튜토리얼 유형           | 대화형                                                    |
| AgentCore 구성 요소    | AgentCore Gateway, AgentCore Identity                     |
| 에이전트 프레임워크      | Strands Agents                                            |
| Gateway Target type  | Smithy                                                   |
| Agent                | Infrastructure agent                                        |
| 인바운드 인증 IdP      | Amazon Cognito                                            |
| Outbound Auth        | IAM                                                   |
| LLM 모델             | Anthropic Claude Sonnet 3.7, Amazon Nova Pro              |
| 튜토리얼 구성 요소      | AgentCore Gateway 생성 및 AgentCore Gateway 호출            |
| 튜토리얼 분야          | 교차 분야                                                 |
| 예제 복잡성           | 쉬움                                                      |
| 사용된 SDK           | boto3                                                     |

튜토리얼의 첫 번째 부분에서는 몇 가지 AmazonCore Gateway 대상을 생성할 것입니다

### 튜토리얼 아키텍처
이 튜토리얼에서는 Amazon S3의 Smithy 사양에 정의된 작업을 MCP 도구로 변환하고 Bedrock AgentCore Gateway에서 호스팅할 것입니다. 이를 통해 사용자는 자신의 AWS 계정에서 Amazon S3와 관련된 모든 질문을 할 수 있습니다.

## 전제 조건

이 튜토리얼을 실행하려면 다음이 필요합니다:
* Jupyter notebook with Python 3.10+
* uv
* AWS 자격 증명
* Amazon Cognito

In [ ]:
# Make sure you download the latest botocore and boto3 libraries.
import shutil
import subprocess
import sys

def ensure_uv_installed():
    if shutil.which("uv") is None:
        print("🔧 'uv' not found. Installing with pip...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", "uv"])
    else:
        print("✅ 'uv' is already installed.")

def uv_install(*packages):
    ensure_uv_installed()
    uv_path = shutil.which("uv")
    print(f"📦 Installing {', '.join(packages)} using uv...")
    subprocess.check_call([uv_path, "pip", "install", *packages])

uv_install("botocore", "boto3")

In [ ]:
import os
# Set AWS credentials if not using SageMaker notebook
os.environ['AWS_ACCESS_KEY_ID'] = ''
os.environ['AWS_SECRET_ACCESS_KEY'] = ''
os.environ['AWS_DEFAULT_REGION'] = 'us-east-1'

In [ ]:
import os
import sys

# 현재 스크립트의 디렉토리 가져오기if '__file__' in globals():
    current_dir = os.path.dirname(os.path.abspath(__file__))
else:
    current_dir = os.getcwd()  # __file__이 정의되지 않은 경우의 대안 (예: Jupyter)

# utils.py가 포함된 디렉토리로 이동 (한 단계 위)utils_dir = os.path.abspath(os.path.join(current_dir, '../..'))

# sys.path에 추가sys.path.insert(0, utils_dir)

# 이제 utils를 가져올 수 있습니다import utils

In [ ]:
#### Gateway가 맡을 IAM 역할 생성import utils

agentcore_gateway_iam_role = utils.create_agentcore_gateway_role_s3_smithy("sample-lambdagateway")
print("Agentcore 게이트웨이 역할 ARN: ", agentcore_gateway_iam_role['Role']['Arn'])

# Gateway에 대한 인바운드 권한 부여를 위한 Amazon Cognito 풀 생성

In [ ]:
# Cognito 사용자 풀 생성import os
import boto3
import requests
import time
from botocore.exceptions import ClientError

REGION = os.environ['AWS_DEFAULT_REGION']
USER_POOL_NAME = "sample-agentcore-gateway-pool"
RESOURCE_SERVER_ID = "sample-agentcore-gateway-id"
RESOURCE_SERVER_NAME = "sample-agentcore-gateway-name"
CLIENT_NAME = "sample-agentcore-gateway-client"
SCOPES = [
    {"ScopeName": "gateway:read", "ScopeDescription": "Read access"},
    {"ScopeName": "gateway:write", "ScopeDescription": "Write access"}
]
scopeString = f"{RESOURCE_SERVER_ID}/gateway:read {RESOURCE_SERVER_ID}/gateway:write"

cognito = boto3.client("cognito-idp", region_name=REGION)

print("Cognito 리소스 생성 또는 검색 중...")
user_pool_id = utils.get_or_create_user_pool(cognito, USER_POOL_NAME)
print(f"사용자 풀 ID: {user_pool_id}")

utils.get_or_create_resource_server(cognito, user_pool_id, RESOURCE_SERVER_ID, RESOURCE_SERVER_NAME, SCOPES)
print("리소스 서버가 확인되었습니다.")

client_id, client_secret  = utils.get_or_create_m2m_client(cognito, user_pool_id, CLIENT_NAME, RESOURCE_SERVER_ID)
print(f"클라이언트 ID: {client_id}")

# 검색 URL 가져오기cognito_discovery_url = f'https://cognito-idp.{REGION}.amazonaws.com/{user_pool_id}/.well-known/openid-configuration'
print(cognito_discovery_url)

# Create the Gateway 

In [ ]:
# CMK 없이 Cognito 권한 부여자로 Gateway 생성. 이전 단계에서 생성한 Cognito 사용자 풀 사용import boto3
gateway_client = boto3.client('bedrock-agentcore-control', region_name = os.environ['AWS_DEFAULT_REGION'])
auth_config = {
    "customJWTAuthorizer": { 
        "allowedClients": [client_id],  # 클라이언트는 Cognito에서 구성된 ClientId와 일치해야 합니다. 예: 7rfbikfsm51j2fpaggacgng84g
        "discoveryUrl": cognito_discovery_url
    }
}
create_response = gateway_client.create_gateway(name='DemoS3Smithyv3',
    roleArn = agentcore_gateway_iam_role['Role']['Arn'], # IAM 역할은 Gateway를 생성/나열/가져오기/삭제할 권한이 있어야 합니다 
    protocolType='MCP',
    authorizerType='CUSTOM_JWT',
    authorizerConfiguration=auth_config, 
    description='AgentCore Gateway with Smithy target'
)
print(create_response)
# GatewayTarget 생성에 사용되는 GatewayID 검색gatewayID = create_response["gatewayId"]
gatewayURL = create_response["gatewayUrl"]
print(gatewayID)

# Create a Smithy target with Amazon S3 spec

We are going to create a smithy gateway target of Amazon S3 apis. We will import the Amazon S3 smithy json file and convert the apis into MCP tools.

#### Upload the S3 Smithy API json file in S3

In [ ]:
# Create an S3 client
session = boto3.session.Session()
s3_client = session.client('s3')
sts_client = session.client('sts')

# Retrieve AWS account ID and region
account_id = sts_client.get_caller_identity()["Account"]
region = session.region_name
# Define parameters
bucket_name = '' # Your s3 bucket to upload the OpenAPI json file.
file_path = 'smithy-specs/s3-apis.json'
object_key = 's3-apis.json'
# Upload the file using put_object and read response
try:
    with open(file_path, 'rb') as file_data:
        response = s3_client.put_object(Bucket=bucket_name, Key=object_key, Body=file_data)

    # Construct the ARN of the uploaded object with account ID and region
    smithy_s3_uri = f's3://{bucket_name}/{object_key}'
    print(f'Uploaded object S3 URI: {smithy_s3_uri}')
except Exception as e:
    print(f'Error uploading file: {e}')

#### Create the gateway target

In [ ]:
# S3 Uri for Smithy spec file
smithy_s3_target_config = {
    "mcp": {
          "smithyModel": {
              "s3": {
                  "uri": smithy_s3_uri
              }
          }
      }
}

# IAM credentials provider configuration
credential_config  = {
        "credentialProviderType" : "GATEWAY_IAM_ROLE"
    }

targetname='DemoSmithytargetForS3'
response = gateway_client.create_gateway_target(
    gatewayIdentifier=gatewayID,
    name=targetname,
    description='Smithy Target with S3Uri using SDK',
    targetConfiguration=smithy_s3_target_config,
    credentialProviderConfigurations=[credential_config])

# Printing the request ID and timestamp for you to report the defects. Please include them while reporting issues/defects  
response_metadata = response['ResponseMetadata']
print(response_metadata)

# Strands Agent에서 Bedrock AgentCore Gateway 호출

Strands 에이전트는 Model Context Protocol(MCP) 사양을 구현하는 Bedrock AgentCore Gateway를 통해 AWS 도구와 원활하게 통합됩니다. 이 통합은 AI 에이전트와 AWS 서비스 간의 안전하고 표준화된 통신을 가능하게 합니다.

핵심적으로 Bedrock AgentCore Gateway는 기본적인 MCP API인 ListTools와 InvokeTools를 노출하는 프로토콜 준수 Gateway 역할을 합니다. 이러한 API를 통해 MCP 호환 클라이언트나 SDK는 안전하고 표준화된 방식으로 사용 가능한 도구를 발견하고 상호 작용할 수 있습니다. Strands 에이전트가 AWS 서비스에 액세스해야 할 때 이러한 MCP 표준화된 엔드포인트를 사용하여 Gateway와 통신합니다.

Gateway의 구현은 (MCP 권한 부여 사양)[https://modelcontextprotocol.org/specification/draft/basic/authorization]을 엄격히 준수하여 강력한 보안과 액세스 제어를 보장합니다. 이는 Strands 에이전트의 모든 도구 호출이 권한 부여 단계를 거치므로 강력한 기능을 활성화하면서 보안을 유지한다는 것을 의미합니다.

예를 들어, Strands 에이전트가 MCP 도구에 액세스해야 할 때 먼저 ListTools를 호출하여 사용 가능한 도구를 발견한 다음 InvokeTools를 사용하여 특정 작업을 실행합니다. Gateway는 필요한 모든 보안 검증, 프로토콜 변환 및 서비스 상호 작용을 처리하여 전체 프로세스를 원활하고 안전하게 만듭니다.

이러한 아키텍처 접근 방식은 MCP 사양을 구현하는 모든 클라이언트나 SDK가 Gateway를 통해 AWS 서비스와 상호 작용할 수 있음을 의미하며, AI 에이전트 통합을 위한 다재다능하고 미래 지향적인 솔루션이 됩니다.

In [ ]:
uv_install("mcp[cli]", "strands-agents")

# 인바운드 권한 부여를 위한 Amazon Cognito에서 액세스 토큰 요청

In [ ]:
print("Amazon Cognito 권한 부여자에서 액세스 토큰 요청 중... 도메인 이름 전파가 완료될 때까지 일정 시간 동안 실패할 수 있습니다")
token_response = utils.get_token(user_pool_id, client_id, client_secret,scopeString,REGION)
token = token_response["access_token"]
print("토큰 응답:", token)

# Ask the IT agent about the s3 resources you have in your AWS account

In [ ]:
from strands.models import BedrockModel
from mcp.client.streamable_http import streamablehttp_client 
from strands.tools.mcp.mcp_client import MCPClient
from strands import Agent

def create_streamable_http_transport():
    return streamablehttp_client(gatewayURL,headers={"Authorization": f"Bearer {token}"})

client = MCPClient(create_streamable_http_transport)

## ~/.aws/credentials에 구성된 IAM 그룹/사용자는 Bedrock 모델에 액세스할 수 있어야 합니다yourmodel = BedrockModel(
    model_id="us.amazon.nova-pro-v1:0",
    temperature=0.7,
)

In [ ]:
from strands import Agent
import logging


# 루트 strands 로거를 구성합니다. 문제를 디버깅하는 경우 DEBUG로 변경하세요.logging.getLogger("strands").setLevel(logging.INFO)

# 로그를 보기 위한 핸들러 추가logging.basicConfig(
    format="%(levelname)s | %(name)s | %(message)s", 
    handlers=[logging.StreamHandler()]
)

with client:
# listTools 호출    tools = client.list_tools_sync()
# 모델과 도구로 에이전트 생성    agent = Agent(model=yourmodel,tools=tools) ## 원하는 모델로 교체할 수 있습니다
    #print(f"에이전트에 로드된 도구는 {agent.tool_names}")
    #print(f"Tools configuration in the agent are {agent.tool_config}")
    # 샘플 프롬프트로 에이전트를 호출합니다. 이는 MCP listTools만 호출하고 LLM이 액세스할 수 있는 도구 목록을 검색합니다. 아래는 실제로 어떤 도구도 호출하지 않습니다.
    agent("Hi , can you list all the s3 buckets. Locate the right tool and call the tool to get the answer")
    #agent("화성 북부 지역의 날씨는 어떤가요?")
# 샘플 프롬프트로 에이전트를 호출하고, 도구를 호출하여 응답을 표시합니다    #agent("주문 ID 123의 주문 상태를 확인하고 도구의 정확한 응답을 보여주세요")
    #MCP 도구를 명시적으로 호출합니다. MCP 도구 이름과 인수는 AWS Lambda 함수 또는 OpenAPI/Smithy API와 일치해야 합니다
    result = client.call_tool_sync(
    tool_use_id="get-insight-weather-1", # 이것을 고유 식별자로 교체할 수 있습니다. 
    name=targetname+"___ListBuckets", # 이는 AWS Lambda 대상 유형을 기반으로 한 도구 이름입니다. 대상 이름에 따라 변경됩니다
    #arguments={"ver": "1.0","feedtype": "json"}
    )
    #Print the MCP Tool response
    print(f"도구 호출 결과: {result['content'][0]['text']}")


# 정리
IAM 역할, IAM 정책, 자격 증명 공급자, AWS Lambda 함수, Cognito 사용자 풀, S3 버킷과 같은 추가 리소스도 생성되며, 정리의 일부로 수동으로 삭제해야 할 수 있습니다. 이는 실행하는 예제에 따라 다릅니다.

## 게이트웨이 삭제 (선택사항)

In [ ]:
import utils
utils.delete_gateway(gateway_client,gatewayID)